In [1]:
import requests
import json
import time
import pandas as pd

from typing import List, Tuple

In [2]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, indent=4)
    print(text)

In [3]:
### Goal:
# connect to new API of laws
#  Rechtsinformationen API v1
# https://api.rechtsinformationsportal.de/v1/docs#operations-top

In [5]:
x = 0
Ri_response = []
Ri_response.append(
    requests.get("https://api.rechtsinformationsportal.de/v1/laws?include=all_fields")
)
next_page = Ri_response[-1].json()["links"]["next"]
while next_page != None:
    Ri_response.append(requests.get(next_page))
    next_page = Ri_response[-1].json()["links"]["next"]
    x += 1

In [8]:
rechtsinformationsportal_list = [
    (Ri_response[page].json()["data"]) for page in range(0, x + 1)
]

In [9]:
with open("../data/source_laws/rechtsinformationsportalAPI.json", "w") as file:
    json.dump(rechtsinformationsportal_list, file)

In [13]:
rechtsinformationsportal_df_list = [
    pd.DataFrame.from_dict(Ri_response[page].json()["data"]) for page in range(0, x + 1)
]

In [14]:
rechtsinformationsportal_df = pd.concat(
    rechtsinformationsportal_df_list, ignore_index=True
)

In [30]:
rechtsinformationsportal_df.head()

,type,id,url,firstPublished,sourceTimestamp,titleShort,titleLong,abbreviation,slug,extraAbbreviations,publicationInfo,statusInfo,notes,attachments
0,law,BJNR172200996,https://api.rechtsinformationsportal.de/v1/law...,1996-11-08,20170822221017,Verordnung zur Begrenzung von Emissionen aus d...,Fünfundzwanzigste Verordnung zur Durchführung\...,25. BImSchV,25_bimschv,[BImSchV 25],"[{'reference': '1996, 1722', 'periodical': 'BG...",[{'comment': 'Neugefasst durch Bek. v. 30.7.20...,"{'body': None, 'footnotes': None, 'documentary...",{}
1,law,BJNR239600014,https://api.rechtsinformationsportal.de/v1/law...,2014-12-22,20150129221520,Beitragssatzverordnung 2015,Verordnung zur Bestimmung der Beitragssätze in...,BSV 2015,bsv_2015,[],"[{'reference': '2014, 2396', 'periodical': 'BG...",[],"{'body': None, 'footnotes': None, 'documentary...",{}
2,law,BJNR097000016,https://api.rechtsinformationsportal.de/v1/law...,2016-04-25,20160609215501,None,Zweite Verordnung zur Durchführung des Finanza...,FinAusglG2011DV 2,finausglg2011dv_2,[],"[{'reference': '2016, 970', 'periodical': 'BGB...",[],"{'body': None, 'footnotes': None, 'documentary...",{}
3,law,BJNR005090911,https://api.rechtsinformationsportal.de/v1/law...,1911-07-19,20180814214502,None,Reichsversicherungsordnung,RVO,rvo,[],"[{'reference': '1911, 509', 'periodical': 'RGB...",[{'comment': 'Zuletzt geändert durch Art. 7 G ...,"{'body': None, 'footnotes': None, 'documentary...",{}
4,law,BJNR078700993,https://api.rechtsinformationsportal.de/v1/law...,1993-06-02,20130405130737,None,Bekanntmachung der Dienstbezüge und Anwärterbe...,BesÜV2Bek 1993-06-02,besuev2bek_1993_06_02,[],"[{'reference': '1993, 787', 'periodical': 'BGB...",[],"{'body': None, 'footnotes': None, 'documentary...",{}


In [15]:
rechtsinformationsportal_df.to_csv(
    index=False, path_or_buf="../data/source_laws/rechtsinformationsportalAPI.csv"
)

In [28]:
rechtsinformationsportal_df.count()

## note that short title is 2476 while title (and the rest) is 6871
## I wanted ot check year rnage but there are som elaws with year 0000
##    and that blocked converting the column as a whole.. so ther eare def data issues..

type                  6871
id                    6871
url                   6871
firstPublished        6871
sourceTimestamp       6871
titleShort            2476
titleLong             6871
abbreviation          6871
slug                  6871
extraAbbreviations    6871
publicationInfo       6871
statusInfo            6871
notes                 6871
attachments           6871
dtype: int64

to request a specific law you need to keep track of the slug so you can use it in the API call

In [37]:
slug = "25_bimschv"

In [38]:
law = requests.get(
    "https://api.rechtsinformationsportal.de/v1/laws/" + slug + "?include=contents"
)

In [42]:
rechtsinformationsportal_df.slug.count()

6871

In [43]:
Ri_slugs_response = []
slug = rechtsinformationsportal_df.slug[0]
for slug in rechtsinformationsportal_df.slug:
    Ri_slugs_response.append(
        requests.get(
            "https://api.rechtsinformationsportal.de/v1/laws/"
            + slug
            + "?include=contents"
        )
    )

ConnectionError: HTTPSConnectionPool(host='api.rechtsinformationsportal.de', port=443): Max retries exceeded with url: /v1/laws/promechg?include=contents (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x12567f0a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [51]:
jprint(Ri_slugs_response[-3].json())

{
    "data": {
        "type": "law",
        "id": "BJNR431500013",
        "url": "https://api.rechtsinformationsportal.de/v1/laws/albeitrbek_2014",
        "firstPublished": "2013-12-19",
        "sourceTimestamp": "20140102220010",
        "titleShort": null,
        "titleLong": "Bekanntmachung der Beitr\u00e4ge und der Beitragszusch\u00fcsse in der Alterssicherung der Landwirte f\u00fcr das Jahr 2014",
        "abbreviation": "ALBeitrBek 2014",
        "slug": "albeitrbek_2014",
        "extraAbbreviations": [],
        "publicationInfo": [
            {
                "reference": "2013, 4315",
                "periodical": "BGBl I"
            }
        ],
        "statusInfo": [],
        "notes": {
            "body": null,
            "footnotes": null,
            "documentaryFootnotes": "<P><BR/> <pre xml:space=\"preserve\">(+++ Textnachweis ab: 20.12.2013 +++) <BR/><BR/></pre></P>"
        },
        "attachments": {},
        "contents": [
            {
               

In [52]:
Ri_slugs_response = [
    (Ri_slugs_response[page].json()["data"]) for page in range(0, x + 1)
]

In [53]:
with open("../data/source_laws/rechtsinformationsportalAPI_laws.json", "w") as file:
    json.dump(Ri_slugs_response, file)

In [54]:
len(Ri_slugs_response)

688

In [66]:
Ri_slugs_response[0]["contents"][0]["body"]

{'type': 'article',
 'id': 'BJNR172200996BJNE000202116',
 'url': 'https://api.rechtsinformationsportal.de/v1/laws/25_bimschv/articles/BJNR172200996BJNE000202116',
 'name': '§ 1',
 'title': 'Anwendungsbereich',
 'parent': None,
 'body': '<P>Diese Verordnung gilt für die Errichtung, die Beschaffenheit und den Betrieb von <DL Font="normal" Type="arabic"><DT>1.</DT><DD Font="normal"><LA Size="normal">Anlagen zur fabrikmäßigen Herstellung von Titandioxid nach dem Sulfat- und Chloridverfahren,</LA></DD><DT>2.</DT><DD Font="normal"><LA Size="normal">Anlagen zum fabrikmäßigen Aufkonzentrieren von Abfallsäuren.</LA></DD></DL></P>',
 'footnotes': None,
 'documentaryFootnotes': None}

In [68]:
import random

In [155]:
slugs = []
for _ in range(30):
    slugs.append(random.choice(rechtsinformationsportal_df.slug))

In [156]:
# assuming the existence of a matching function,
# which can match the title of the law, and return a slug using the above json or table


def get_source_law_rechtsinformationsportal(slug: str) -> List[dict]:
    try:
        ri_law_api = requests.get(
            "https://api.rechtsinformationsportal.de/v1/laws/"
            + slug
            + "?include=contents"
        )
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    contents = ri_law_api.json()["data"]["contents"]

    keys = {
        "type",
        "id",
        "name",
        "title",
        "parent",
        "body",
        "footnotes",
    }
    source_law = []
    for item in contents:
        source_law.append({key: item[key] for key in keys if key in item})

    return source_law

In [165]:
law = get_source_law_rechtsinformationsportal("besuev2bek_1993_06_02")

In [169]:
law[-1]

{'type': 'article',
 'id': 'BJNR078700993BJNE001400307',
 'parent': None,
 'name': 'Anlage IV D',
 'body': '<P><kommentar typ="Fundstelle">(Fundstelle des Originaltextes: BGBl. I 1993, 808 - 810)</kommentar><BR/><table frame="all" tocentry="%yes;"><tgroup align="left" char="" charoff="50" cols="4" colsep="0" rowsep="0"><colspec colname="col1" colnum="1" colwidth="100"/><colspec colname="col2" colnum="2" colwidth="280"/><colspec colname="col3" colnum="3" colwidth="100"/><colspec align="right" colname="col4" colnum="4" colwidth="150"/><tbody valign="top"><row rowsep="1"><entry VJ="1" colsep="1" nameend="col2" namest="col1">Dem Grunde nach geregelt in</entry><entry VJ="1" nameend="col4" namest="col3">Betrag in Deutscher Mark,<BR/>Vomhundert, Bruchteil</entry></row><row><entry VJ="1" nameend="col4" namest="col1">Bundesbesoldungsgesetz</entry></row><row><entry VJ="1">§ 44</entry><entry VJ="1">\xa0</entry><entry VJ="1">bis zu</entry><entry VJ="1">148,00</entry></row><row><entry VJ="1">§ 48 A

In [157]:
source_laws = []
for slug in slugs:
    source_laws = get_source_law_rechtsinformationsportal(slug)

In [162]:
source_laws[0]

{'type': 'article',
 'id': 'BJNR393300021BJNE000100000',
 'parent': None,
 'name': 'Inhaltsübersicht',
 'body': '<TOC><Ident Class="S2">Kapitel 1</Ident><P/><Title Align="auto" Class="S2">Allgemeine Vorschriften</Title><P/><table colsep="0" frame="none" pgwide="1" rowsep="0" tocentry="%yes;"><tgroup align="left" char="" charoff="50" cols="2"><colspec colname="col1" colwidth="10*"/><colspec colname="col2" colwidth="70*"/><tbody valign="top"><row><entry VJ="1" colname="col1">§ \xa01</entry><entry VJ="1" colname="col2">(zukünftig in Kraft)</entry></row><row><entry VJ="1" colname="col1">§ \xa02</entry><entry VJ="1" colname="col2">(zukünftig in Kraft)</entry></row><row><entry VJ="1" colname="col1">§ \xa03</entry><entry VJ="1" colname="col2">(zukünftig in Kraft)</entry></row><row><entry VJ="1" colname="col1">§ \xa04</entry><entry VJ="1" colname="col2">(zukünftig in Kraft)</entry></row><row><entry VJ="1" colname="col1">§ \xa05</entry><entry VJ="1" colname="col2">(zukünftig in Kraft)</entry></